# AI Toolkit by Ostris
## Memecoin Multi-Concept LoRA Training


In [ ]:
!nvidia-smi


In [ ]:
import os
import subprocess

if os.path.exists('doom-ai-toolkit'):
    subprocess.run(['git', '-C', 'doom-ai-toolkit', 'pull', 'origin', 'main'], check=True)
    print('Repository updated successfully')
else:
    subprocess.run(['git', 'clone', 'https://github.com/posaune0423/doom-ai-toolkit.git'], check=True)
    print('Repository cloned successfully')

This notebook uses the dataset from the git repository. The memecoin dataset is already included in the `dataset/` folder at the repo root, so no extra setup is required.


In [ ]:
!cd doom-ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt

In [ ]:
import os

!pip install --quiet --force-reinstall --no-deps numpy==1.26.3

os.kill(os.getpid(), 9)


## Model License
FLUX.1-schnell is Apache 2.0 and does not require a gated Hugging Face token, but you still need to agree to the model terms on Hugging Face and keep a token handy if you plan to access private models. Use the next cell to set `HF_TOKEN` when needed.


In [ ]:
import os

SECRET_NAME = "HF_TOKEN"

from google.colab import userdata  # type: ignore

hf_token = userdata.get(SECRET_NAME)
if not hf_token:
    raise RuntimeError(
        f"Failed to retrieve required Hugging Face token from Colab secret '{SECRET_NAME}'."
    )

os.environ["HF_TOKEN"] = hf_token
print(f"HF_TOKEN environment variable has been set from Colab secret '{SECRET_NAME}'.")


In [ ]:
import os
import sys
sys.path.append('/content/doom-ai-toolkit')
from toolkit.job import run_job
import yaml
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"


## Setup

This is your config. It is documented pretty well. The configuration is loaded from a YAML file. This will run as is without modification, but feel free to edit as you want.


In [ ]:
from pathlib import Path

config_path = Path('/content/doom-ai-toolkit/config/train_memecoin_pack_lora.yaml')
job_to_run = yaml.safe_load(config_path.read_text())
print('Configuration loaded')
print(f"Training: {job_to_run['config']['name']}")
print(f"Datasets: {len(job_to_run['config']['process'][0]['datasets'])}")


## Run it

Below does all the magic. Check your folders to the left. Items will be in output/memecoin_multi_lora_v1. In the samples folder, there are periodic samples. This doesn't work great with colab. They will be in /content/doom-ai-toolkit/output


In [ ]:
run_job(job_to_run)


## Done

Check your output dir and get your LoRA


In [ ]:
from IPython.display import Image, display
import os
from pathlib import Path

# ベースのoutputディレクトリ（環境に合わせて必要なら変更）
base_output_dir = Path('/content/doom-ai-toolkit/output')

# output配下の「*/samples」ディレクトリを全て集めて、更新日時が新しいものを選ぶ
sample_dirs = [p for p in base_output_dir.glob('*/samples') if p.is_dir()]
if sample_dirs:
    samples_dir = sorted(sample_dirs, key=lambda p: p.stat().st_mtime)[-1]
    print(f'Using samples_dir: {samples_dir}')

    images = sorted([f for f in os.listdir(samples_dir) if f.endswith('.png')])
    for img in images[-6:]:
        display(Image(filename=os.path.join(samples_dir, img)))
else:
    print(f'No samples directory found under {base_output_dir}')
